In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [4]:
%%writefile app.py
import streamlit as st
import uuid
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize Streamlit Page
st.set_page_config(page_title="AI Conversational Data Science Tutor")

# Title and Description
st.markdown("<h2 style='text-align: center;'>🤖 AI Conversational Data Science Tutor</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center;'>Ask me anything about Data Science!</p>", unsafe_allow_html=True)

# Generate a unique session ID for the user
if "session_id" not in st.session_state:
    st.session_state.session_id = str(uuid.uuid4())

# Function to get session message history from DB
def get_session_message_history_from_db(session_id):
    return SQLChatMessageHistory(
        session_id=session_id,
        connection="sqlite:////content/drive/MyDrive/Innomatics/sqlite.db"  # Ensure path is correct
    )

# Initialize chat history
chat_history = get_session_message_history_from_db(st.session_state.session_id)

# Load chat history from DB if not already in session
if "messages" not in st.session_state:
    st.session_state.messages = chat_history.messages  # Retrieve stored messages

# Initialize the chatbot
chat_model = ChatGoogleGenerativeAI(api_key="api-key", model="gemini-2.0-flash-exp")
output_parser = StrOutputParser()

chat_template = ChatPromptTemplate(
    messages=[
        ("system", """You are a helpful AI Tutor for Data Science.
        Students will ask you doubts related to various topics in data science.
        You are expected to reply in as much detail as possible.
        Use examples while explaining a concept.
        If a student asks a question outside the data science domain,
        politely decline and tell them to ask a relevant question."""),

        MessagesPlaceholder(variable_name="history"),  # Preserves chat history
        ("human", "{human_input}")
    ]
)

chain = chat_template | chat_model | output_parser

conversation_chain = RunnableWithMessageHistory(
    chain,
    get_session_message_history_from_db,
    input_messages_key="human_input",
    history_messages_key="history"
)

# Function to get chatbot response
def chat_bot(session_id, prompt):
    config = {"configurable": {"session_id": session_id}}
    input_prompt = {"human_input": prompt}
    response = conversation_chain.invoke(input_prompt, config=config)

    # Store message history in DB
    chat_history.add_user_message(prompt)
    chat_history.add_ai_message(response)

    return response

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# User input field
user_input = st.chat_input("Ask a data science question...")

if user_input:
    # Append user message to session history
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # Get AI response
    response = chat_bot(st.session_state.session_id, user_input)

    # Append AI response to session history
    st.session_state.messages.append({"role": "assistant", "content": response})
    with st.chat_message("assistant"):
        st.markdown(response)


Writing app.py


In [5]:
!wget -q -O - ipv4.icanhazip.com

34.67.196.215


In [7]:
!streamlit run app.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴⠦⠧⠇
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.67.196.215:8501

⠏your url is: https://tidy-suns-return.loca.lt
  Stopping...
^C
